In [1]:
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
import torch.backends.cudnn as cudnn

from tqdm.notebook import tqdm

In [2]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
class GasDataset(Dataset):
    def __init__(self, df, seq_len, year=None):
        self.seq_len = seq_len
        self.df = df
        
        if year is not None:
            self.df = self.df.loc[self.df['year'] == year]
    
    def __len__(self):
        return len(self.df) - self.seq_len - 1

    def __getitem__(self, idx):
        data = self.df.iloc[idx:idx + self.seq_len]
        x = torch.from_numpy(np.array(data[['supply']], dtype=np.float32))
        y = torch.from_numpy(np.array(self.df.iloc[idx + self.seq_len, -4], dtype=np.float32))

        return x, y

In [4]:
class TensorExtractor(nn.Module):
    def forward(self, x):
        tensor, _ = x
        return tensor
    
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layer, conv_size=None):
        super(LSTM, self).__init__()
        lstm_layer = []
        if conv_size is not None:
            conv = nn.Conv1d(input_size, conv_size, kernel_size=4, stride=2, padding=0)
            lstm_layer.append(conv)
        lstm_layer.append(nn.LSTM(input_size, hidden_size, num_layer))
        lstm_layer.append(TensorExtractor())
        lstm_layer.append(nn.Linear(hidden_size, 1))
        self.lstm = nn.Sequential(*lstm_layer)
    
    def forward(self, x):
        return self.lstm(x)

In [5]:
def train(model, train_loader, val_data, optimizer, criterion, epoch):
    model.train()
    for i in tqdm(range(epoch)):
        for batch_idx, (x, y) in enumerate(train_loader):
            x = x.squeeze(-1).to(device)
            y = y.unsqueeze(-1).to(device)
            
            optimizer.zero_grad()
            output = model(x)
            
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

        if i % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i, epoch,
                100. * i / epoch, loss.item()))

    if val_data is not None:
        model.eval()
        loss = 0
        true_val = []
        predictions = []
        with torch.no_grad():
            for batch_idx, (x, y) in enumerate(val_data):
                
                x = x.view(1, -1).to(device)
                y = y.unsqueeze(-1).to(device)
                output = model(x).squeeze(-1)

                loss += criterion(output, y)
                true_val.append(float(y.cpu().numpy()))
                predictions.append(float(output.cpu().numpy()))
        

        print('\nTest set: Average loss: {:.4f}'.format(loss / len(val_data)))
        # print(true_val, predictions)
        plt.figure(figsize=(30,10))
        x = np.arange(len(true_val))
        plt.plot(x, true_val, label='true', c='blue')
        plt.plot(x, predictions, label='predictions', c='red')

        

In [6]:
type = 'A'
seq_len = 7
train_rate = 0.7
df = pd.read_csv("data/kor_gas_day.csv")
train_len = int(len(df) * train_rate)

df = df.loc[df['type'] == type]
df_train, df_val = df.iloc[:train_len], df.iloc[train_len:]
train_dataset = GasDataset(df_train, seq_len)
val_dataset = GasDataset(df_val, seq_len)

In [7]:
hidden_szie = 128
num_layers = 20
learning_rate = 0.0001
epochs = 300
batch_szie = 32

model = LSTM(seq_len, hidden_szie, num_layers).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss().to(device)

train_loader = DataLoader(train_dataset, batch_size=batch_szie, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_szie)

In [8]:
train(model, train_loader, val_dataset, optimizer, criterion, epochs)

  0%|          | 0/300 [00:00<?, ?it/s]

Train Epoch: 300 [0/300 (0%)]	Loss: 1095078.875000
Train Epoch: 300 [100/300 (33%)]	Loss: 2558193.750000
Train Epoch: 300 [200/300 (67%)]	Loss: 1488681.500000


ValueError: __len__() should return >= 0

In [ ]:
loss = 0
true_val = []
predictions = []

for x, y in train_dataset:
    print(x, y)
    true_val.extend(list(float(x.squeeze().cpu().numpy())))

plt.figure(figsize=(30,10))
x = np.arange(len(true_val))
plt.plot(x, true_val, label='true', c='blue')

AttributeError: 'numpy.int64' object has no attribute 'loc'